In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm
from sklearn.metrics import ConfusionMatrixDisplay
from xgboost import XGBClassifier

In [1]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/SNU Big Data Fintec/Lending Club/Macro/data_macro_selected.csv')

# 데이터 전처리

def convert_emp_length(emp_length):
    if emp_length == '< 1 year':
        return 0
    elif emp_length == '10+ years':
        return 10
    else:
        return int(emp_length.split()[0])

data['emp_length'] = data['emp_length'].apply(convert_emp_length)

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

data_real_number_columns = data.select_dtypes(include = ['float']).columns
data_real_number = data[data_real_number_columns]

#스케일링
scaler = MinMaxScaler()
data_real_number_normalized = scaler.fit_transform(data_real_number)
data_real_number_normalized_df = pd.DataFrame(data_real_number_normalized, columns=data_real_number_columns)

categorical_columns = data.select_dtypes(include=['object']).columns # 범주형 변수만을 선택
one_hot_encoder = OneHotEncoder() # OneHotEncoder 객체 생성
dummy_variables = one_hot_encoder.fit_transform(data[categorical_columns]) # 범주형 변수를 더미 변수로 변환
dummy_df = pd.DataFrame(dummy_variables.toarray(), columns=one_hot_encoder.get_feature_names_out(categorical_columns)) # 더미 변수를 데이터프레임으로 변환


# 실수형 자료 + 더미변수 // (원래의 데이터프레임에서 범주형 변수를 제거하고 더미 변수 데이터프레임을 추가)
data_final = pd.concat([data_real_number_normalized_df, dummy_df], axis=1)

# 부도자 첨가하기
add_column = data['defaulter']
data_final['defaulter'] = add_column

x = data_final.drop(columns='defaulter', axis=1)
y = data_final[['defaulter']]

In [20]:
import numpy as np

x_arr=np.asarray(x)
y_arr=np.asarray(y)

from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5)

def clf():
    rfc = XGBClassifier(random_state=None)
    return rfc

In [23]:
col_list=x.columns


from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve

a_rfc=[] # accuracy
r_rfc=[] # recall
p_rfc=[] # precision
f_rfc=[] # f-score
ro_rfc=[] # roc

#for 문 안에서 이전에 설정한 n_split=5의 값으로 데이터를 split해 줍니다.
for train_index, test_index in kf.split(x_arr, y_arr):

    x_train, x_test = x_arr[train_index], x_arr[test_index]
    y_train, y_test = y_arr[train_index], y_arr[test_index]
    x_test=pd.DataFrame(x_test, columns=col_list)
    # array concat
    y_train2=y_train.reshape(y_train.shape[0],1)
    trset=np.concatenate([x_train,y_train2],axis=1)

    # over sampling
    # 여기서는 중복추출을 통해 단순히 minority class 데이터를 2배로 뻥튀기 시키겠습니다.
    # 원하신다면, 이 부분에는 smote관련 코드를 넣어도 됩니다.

    a = list(col_list)
    a.append('defaulter')
    trset_df=pd.DataFrame(trset, columns=a)

    x_go = trset_df.drop(['defaulter'],axis=1)
    y_go = trset_df['defaulter']

    from imblearn.over_sampling import SMOTE
    smote = SMOTE(random_state=None)
    x_over, y_over = smote.fit_resample(x_go, y_go)

    #스플릿
    from sklearn.model_selection import train_test_split
    x_train_fin, x_test, y_train_fin, y_test = train_test_split(x_over, y_over, test_size=0.2, random_state=None)

    #clf
    rfc = clf()
    rfc.fit(x_train_fin, y_train_fin)
    pred_rfc = rfc.predict(x_test)

    #score
    accuracy_rfc = accuracy_score(y_test,pred_rfc)
    a_rfc.append(accuracy_rfc)

    recall_rfc = recall_score(y_test,pred_rfc)
    r_rfc.append(recall_rfc)

    precision_rfc = precision_score(y_test,pred_rfc)
    p_rfc.append(precision_rfc)

    f1_rfc = f1_score(y_test,pred_rfc)
    f_rfc.append(f1_rfc)

    roc_rfc = roc_curve(y_test,pred_rfc)
    ro_rfc.append(roc_rfc)

In [24]:
#5번 시험본 결과에 대한 평균

print(np.mean(a_rfc), np.mean(p_rfc), np.mean(r_rfc), np.mean(f_rfc), np.mean(ro_rfc))

0.8792869411819322 0.9739600930903378 0.7794082990481263 0.8658903150300675 0.6444718150866534


In [25]:
'''
정확도 : 0.8792
pre : 0.97396
리콜 : 0.7794082
f1 : 0.865890
'''

'\n정확도 : 0.8792\npre : 0.97396\n리콜 : 0.7794082\nf1 : 0.865890\n'

In [3]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/SNU Big Data Fintec/Lending Club/Macro/data_macro_selected.csv')

# 데이터 전처리

def convert_emp_length(emp_length):
    if emp_length == '< 1 year':
        return 0
    elif emp_length == '10+ years':
        return 10
    else:
        return int(emp_length.split()[0])

data['emp_length'] = data['emp_length'].apply(convert_emp_length)

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

data_real_number_columns = data.select_dtypes(include = ['float']).columns
data_real_number = data[data_real_number_columns]

#스케일링
scaler = MinMaxScaler()
data_real_number_normalized = scaler.fit_transform(data_real_number)
data_real_number_normalized_df = pd.DataFrame(data_real_number_normalized, columns=data_real_number_columns)

categorical_columns = data.select_dtypes(include=['object']).columns # 범주형 변수만을 선택
one_hot_encoder = OneHotEncoder() # OneHotEncoder 객체 생성
dummy_variables = one_hot_encoder.fit_transform(data[categorical_columns]) # 범주형 변수를 더미 변수로 변환
dummy_df = pd.DataFrame(dummy_variables.toarray(), columns=one_hot_encoder.get_feature_names_out(categorical_columns)) # 더미 변수를 데이터프레임으로 변환


# 실수형 자료 + 더미변수 // (원래의 데이터프레임에서 범주형 변수를 제거하고 더미 변수 데이터프레임을 추가)
data_final = pd.concat([data_real_number_normalized_df, dummy_df], axis=1)

# 부도자 첨가하기
add_column = data['defaulter']
data_final['defaulter'] = add_column

X = data_final.drop(columns='defaulter', axis=1)
y = data_final[['defaulter']]

# 오버샘플링, stratifiedkfold(클래스비율유지), grid or randomsearch

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

stk = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)

for train_index, test_index in stk.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

xgb = XGBClassifier(random_state=None)

oversampling_accuracy_lst = []
oversampling_precision_lst = []
oversampling_recall_lst = []
oversampling_f1_lst = []
oversampling_auc_lst = []

for train, test in stk.split(original_Xtrain, original_ytrain):
  pipeline = imbalanced_make_pipeline(SMOTE(sampling_strategy='minority'), xgb)
  model = pipeline.fit(original_Xtrain[train], original_ytrain[train])
  prediction = model.predict(original_Xtrain[test])

  oversampling_accuracy_lst.append(pipeline.score(original_Xtrain[test], original_ytrain[test]))
  oversampling_precision_lst.append(precision_score(original_ytrain[test], prediction))
  oversampling_recall_lst.append(recall_score(original_ytrain[test], prediction))
  oversampling_f1_lst.append(f1_score(original_ytrain[test], prediction))
  oversampling_auc_lst.append(roc_auc_score(original_ytrain[test], prediction))

Train: [     0      2      3 ... 837979 837980 837981] Test: [     1     11     19 ... 837969 837971 837977]
Train: [     1      2      3 ... 837979 837980 837981] Test: [     0      7      8 ... 837958 837970 837975]
Train: [     0      1      2 ... 837979 837980 837981] Test: [     9     14     18 ... 837973 837974 837976]
Train: [     0      1      7 ... 837978 837979 837981] Test: [     2      3      4 ... 837965 837972 837980]
Train: [     0      1      2 ... 837976 837977 837980] Test: [    12     16     17 ... 837978 837979 837981]


In [5]:
import numpy as np
print(np.mean(oversampling_accuracy_lst), np.mean(oversampling_precision_lst), np.mean(oversampling_recall_lst), np.mean(oversampling_f1_lst), np.mean(oversampling_auc_lst))

0.8067695331331594 0.5456645586294517 0.10532979803348444 0.17656905459663766 0.5419262878905903


In [ ]:
####